### 1- Create stats masks - Electrodes to be plotted
    Elec that reach the threshold in time (min sig)

In [23]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
from brainpipe.statistics import *
import numpy as np
from itertools import product

phase = 'odor'
conds = ['Low','High']
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/LDA_PAC_E_EpiPerf_LowHigh_BBG/')
path_to_save = join(path_npz, 'masks_stat/')
f_form = '{}_sources_{}_{}_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
f_form = join(path_npz, f_form)
f_save = '{}_mask_stat_{}_minwin{}_th{}.npy'
f_save = join(path_to_save,f_save)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
#freqs = ['2_theta', '3_alpha','4_beta','5_gamma1','6_gamma2']
freqs = ['0_theta', '1_alpha']
th = 0.01
n_perms = 1000
min_win = 1.

rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG',
                'SFG','pPirT']

#Define parameters
mat = np.load(f_form.format('All_subjects','0_theta',phase))
print(mat.files,mat['s_labels'][:10])
idx_rois = np.where([roi in rois_to_keep for roi in mat['s_labels']])
s_su = mat['su_codes'][idx_rois]
subjects = np.unique(s_su)
print('subjects',subjects)

#Find threshold by subject
th_by_su = []
for su in sorted(subjects):
    th_su = []
    for freq in freqs:
        mat = np.load(f_form.format('All_subjects',freq,phase))
        id_su = np.where(mat['su_codes'][idx_rois]==su)
        s_perm = mat['s_perm'][idx_rois][id_su]
        th_elecs = []
        for elec in range(s_perm.shape[0]):
            #print(s_perm.shape, s_perm[elec].shape, s_perm[elec].size)
            if s_perm[elec].size > n_perms:
                perm = s_perm[elec]#[:,10:30]
            else:
                perm = s_perm[elec][:,np.newaxis]
            th_perm = perm_pvalue2level(perm, p=th, maxst=True)[0]
            th_elecs.append(th_perm)
        th_su.append(max(th_elecs))
    max_su = max(th_su)
    th_by_su.append(max_su)
print('perm max size',th_by_su)

-> Olfacto loaded
['s_xyz', 's_da', 's_labels', 's_elec_pow1', 's_elec', 'su_codes', 's_BA', 's_aal_RL', 's_aal', 's_elec_pow0', 's_perm', 's_channels'] ['SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG']
subjects ['S0' 'S1' 'S2' 'S3' 'S4' 'S5']
perm max size [0.81666666666666676, 0.76666666666666683, 0.67047619047619045, 0.80000000000000004, 0.68333333333333335, 0.81666666666666676]


In [25]:
from collections import Counter
# Create masks for a specific perm threshold
for freq in freqs:
    mat = np.load(f_form.format('All_subjects',freq,phase))    
    mask_f, labels = np.array([]),[]
    for i,su in enumerate(sorted(subjects)):
        id_su = np.where(mat['su_codes'][idx_rois]==su)
#         print('before select ROIs',mat['s_da'].shape, np.size(mat['s_da'][1]))
        if np.size(mat['s_da'][1]) > 1:
            s_da = mat['s_da'][idx_rois][id_su]
        else:
            s_da = mat['s_da'][idx_rois][id_su][:,np.newaxis]
        #print('after select ROIs',s_da.shape)
        s_labels = mat['s_labels'][idx_rois][id_su]
        s_channels = mat['s_channels'][idx_rois][id_su]
        #print('elecs sel',s_labels.shape)
        n_elecs, npts = s_da.shape
        mask_su = []
        for elec in range(n_elecs):
            da_elec = s_da[elec]
            label = s_labels[elec]
            channel = s_channels[elec]
            pvals = np.array([])
            for t in range(npts):
                if da_elec[t] > th_by_su[i]:
                    pval = 1
                    labels.append(label)
                if da_elec[t] <= th_by_su[i]:
                    pval = 0
                pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
            underp = np.where(pvals == 1)[0]
            pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
            signif = [True for k in pvsplit if len(k) >= min_win]
            if len(signif) >=1:
                remove = False
                print(su,label,channel)
            else:
                remove = True
            mask_su.append(remove)
        mask_f = np.hstack((mask_f, mask_su)) if np.size(mask_f) else mask_su
    print(freq, len(mask_f),len(np.where(mask_f == False)[0]),Counter(labels))
    np.save(f_save.format('All_subjects',freq,min_win,th,'freqs'),mask_f)

0_theta 282 0 Counter()
S2 IFG o9-o8
S3 OFC o8-o7
S4 OFC o9-o8
1_alpha 282 3 Counter({'OFC': 2, 'IFG': 1})
